In [ ]:
## List of experiments to run
## 1] Default scenario
## 2] flashattention
## 3] expandable_segments: Set expandable segments -- check if it works in notebook
## 4] flash_expandable: flashattention + expandable segments
## 5] torch.compile
## 6] sdpa
## 7] sdpa_torch.compile: sdpa + torch.compile
## 8] 8_bit: 8-bit quantization

In [ ]:
prompts_for_experiment = ["What is the firm period for Ognibene?", "What is the firm period for Hengli?", "What are the material planning requirements for Ognibene?", "What are the material planning requirements for Hengli?", "What are the warranty requirements for Hengli?", "Can you create a table showing the warranty requirements for Ognibene and Hengli?"]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig  
import torch
from transformers.agents import ReactCodeAgent
import pandas as pd
import HF_TOKEN
import time
import os
import gc
from torch.nn.attention import SDPBackend, sdpa_kernel

In [ ]:
contract_file_full = "/home/vp899/projects/Agent_System/Input/Contracts/Full_Contracts_Consol.md"

In [ ]:
with open(contract_file_full, 'r') as file_contract_full:
    input_text_contract_full = file_contract_full.read()

In [ ]:
system_prompt_contract_asst = "You are a helpful digital assistant. You will provide clear and concise answers on the input text you have been provided. You must answer in complete sentences. The input text is enclosed within <Input> and </Input>. The input text contains information on contracts with suppliers. Each individual supplier contract is enclosed within tags <Contract Between {Vendor Name} and CNH Industrial Italia SpA> and </Contract Between {Vendor Name} and CNH Industrial Italia SpA>. For example the contract information with Wipro Enterprises (P) Limited would be enclosed between the tags </Contract Between Wipro Enterprises (P) Limited and CNH Industrial Italia SpA> and </Contract Between Wipro Enterprises (P) Limited and CNH Industrial Italia SpA>. At the beginning of the contract text, there are also tags specifying the supplier name. \n <Input> \n" +  input_text_contract_full + "\n </Input>"

In [ ]:
prompt_for_chat = [{"role": "system", "content": system_prompt_contract_asst}]

In [ ]:
model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"
llama31_hf_token = HF_TOKEN.HF_TOKEN

In [ ]:
## set the experiment to run
# experiment_to_run = "default"
experiment_to_run = "expandable_segments"
experiment_to_run = "sdpa"
experiment_to_run = "torch.compile"
experiment_to_run = "torch.compile_dynamic_mode"
experiment_to_run = "torch.compile_model"

In [ ]:
if experiment_to_run == "expandable_segments":
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# if experiment_to_run != default then set os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
if experiment_to_run != "default":
    os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
# print the environment variable
print(os.environ["PYTORCH_CUDA_ALLOC_CONF"])

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token = llama31_hf_token)
terminators = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<|eot_id|>")]

In [ ]:
# experiment_list = ["flashattention_expandable", "torch.compile", "sdpa", "torch.compile_dynamic_mode", "sdpa_torch.compile", "8_bit", "8_bit_flash"]
experiment_list = ["flashattention_expandable", "torch.compile", "sdpa", "sdpa_torch.compile", "8_bit", "8_bit_flash"]
experiment_list = ["default", "expandable_segments", "flashattention_expandable", "8_bit", "8_bit_flash"]

In [ ]:
if experiment_to_run == "default":
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token,)
if experiment_to_run == "expandable_segments":
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token,)
if experiment_to_run == "sdpa":
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token,)
if experiment_to_run == "8_bit":
    quantization_config = BitsAndBytesConfig(load_in_8bit=True)
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token, attn_implementation="flash_attention_2", quantization_config=quantization_config)
if experiment_to_run == "torch.compile":
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token,)
    model.generation_config.cache_implementation = "static"
    model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)
if experiment_to_run == "torch.compile_dynamic_mode":
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token,)
    model.generation_config.cache_implementation = "static"
    model.forward = torch.compile(model.forward, mode="reduce-overhead", dynamic=True)
if experiment_to_run == "torch.compile_model":
    model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token,)
    model.generation_config.cache_implementation = "static"
    model = torch.compile(model)


In [ ]:
# create a dataframe with columns: Experiment_Name, Prompt, Context_Length, pre_gen_max_mem_allocated_gpu0, pre_gen_max_mem_allocated_gpu1, pre_gen_reserved_mem_gpu0, pre_gen_reserved_mem_gpu1, post_gen_max_mem_allocated_gpu0, post_gen_max_mem_allocated_gpu1, post_gen_reserved_mem_gpu0, post_gen_reserved_mem_gpu1, latency, llm_answer
experiment_results_df = pd.DataFrame(columns = ["Experiment_Name", "Prompt", "Context_Length", "pre_gen_max_mem_allocated_gpu0", "pre_gen_max_mem_allocated_gpu1", "pre_gen_reserved_mem_gpu0", "pre_gen_reserved_mem_gpu1", "post_gen_max_mem_allocated_gpu0", "post_gen_max_mem_allocated_gpu1", "post_gen_reserved_mem_gpu0", "post_gen_reserved_mem_gpu1", "latency", "llm_answer"])

In [ ]:
batch_experiment = 1
if batch_experiment == 1:
    for experiment_name in experiment_list:
        print("Running experiment: ", experiment_name)
        if experiment_name == "flashattention":
            model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token, attn_implementation="flash_attention_2",)
        elif experiment_name == "torch.compile":
            model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token,)
            model.generation_config.cache_implementation = "static"
            model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)
        elif experiment_name == "sdpa":
            model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token,)
        elif experiment_name == "sdpa_torch.compile":
            try:
                model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token,)
                model.generation_config.cache_implementation = "static"
                model.forward = torch.compile(model.forward, mode="reduce-overhead", fullgraph=True)
            except:
                print("Error in running sdpa_torch.compile")            
        elif experiment_name == "torch.compile_dynamic_mode":
            model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token,)
            model.generation_config.cache_implementation = "static"
            model.forward = torch.compile(model.forward, mode="reduce-overhead", dynamic=True)
        elif experiment_to_run == "8_bit":
            quantization_config = BitsAndBytesConfig(load_in_8bit=True)
            model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token,  quantization_config=quantization_config)
        elif experiment_to_run == "8_bit_flash":
            quantization_config = BitsAndBytesConfig(load_in_8bit=True)
            model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto", token = llama31_hf_token, attn_implementation="flash_attention_2", quantization_config=quantization_config)
        print("     Model loaded for experiment: ", experiment_name)
        for prompt in prompts_for_experiment:
            # print experiment_name and prompt with prefix "Running experiment: " in a single line
            print("             Running experiment: ", experiment_name, " with prompt: ", prompt)

            pre_gen_max_mem_allocated_gpu0 = "{:.2f}".format(torch.cuda.max_memory_allocated(0)/1024**3) 
            pre_gen_max_mem_allocated_gpu1 = "{:.2f}".format(torch.cuda.max_memory_allocated(1)/1024**3)
            mem_stats_gpu0 = torch.cuda.memory_stats(0)
            mem_stats_gpu1 = torch.cuda.memory_stats(1)
            pre_gen_reserved_mem_gpu0 = "{:.2f}".format(mem_stats_gpu0["reserved_bytes.all.peak"]/1024**3)
            # pre_gen_active_mem_gpu0 = "{:.2f}".format(mem_stats_gpu0["active_bytes.all.peak"]/1024**3)
            pre_gen_reserved_mem_gpu1 = "{:.2f}".format(mem_stats_gpu1["reserved_bytes.all.peak"]/1024**3)
            prompt_for_chat.append({"role": "user", "content": prompt})    
            input_ids = tokenizer.apply_chat_template(prompt_for_chat, add_generation_prompt=True, return_tensors="pt").to(model.device)
            start_time = time.time() 
            # if experiment name is sdpa or sdpa_torch.compile, use the sdpa_kernel context manager
            if experiment_name == "sdpa" or experiment_name == "sdpa_torch.compile":
                with sdpa_kernel(SDPBackend.FLASH_ATTENTION):
                    outputs = model.generate(input_ids, max_new_tokens=256, eos_token_id=terminators, do_sample=True, temperature=0.6, top_p=0.9,)
            else:
                outputs = model.generate(input_ids, max_new_tokens=256, eos_token_id=terminators, do_sample=True, temperature=0.6, top_p=0.9,)
            end_time = time.time()
            latency = end_time - start_time
            response =  outputs[0][input_ids.shape[-1]:]
            llm_answer = tokenizer.decode(response, skip_special_tokens=True)
            prompt_for_chat.append({"role": "assistant", "content": llm_answer})
            post_gen_max_mem_allocated_gpu0 = "{:.2f}".format(torch.cuda.max_memory_allocated(0)/1024**3)
            post_gen_max_mem_allocated_gpu1 = "{:.2f}".format(torch.cuda.max_memory_allocated(1)/1024**3)
            mem_stats_gpu0 = torch.cuda.memory_stats()
            mem_stats_gpu1 = torch.cuda.memory_stats(1)
            post_gen_reserved_mem_gpu0 = "{:.2f}".format(mem_stats_gpu0["reserved_bytes.all.peak"]/1024**3)
            post_gen_reserved_mem_gpu1 = "{:.2f}".format(mem_stats_gpu1["reserved_bytes.all.peak"]/1024**3)
            # add the information to the dataframe using loc
            experiment_results_df.loc[len(experiment_results_df)] = ["default", prompt, input_ids.shape[-1], pre_gen_max_mem_allocated_gpu0, pre_gen_max_mem_allocated_gpu1, pre_gen_reserved_mem_gpu0, pre_gen_reserved_mem_gpu1, post_gen_max_mem_allocated_gpu0, post_gen_max_mem_allocated_gpu1, post_gen_reserved_mem_gpu0, post_gen_reserved_mem_gpu1, latency, llm_answer]
        del model
        gc.collect()
        torch.cuda.empty_cache()
        torch.cuda.reset_peak_memory_stats()
elif batch_experiment == 0:
    prompts_for_experiment = ["What is the firm period for Ognibene?"]
    # print the experiment_to_run with prefix "Running experiment: "
    print("Running experiment: ", experiment_to_run)
    # print experiment_to_run using {experiment_to_run}
    

    for prompt in prompts_for_experiment:
        pre_gen_max_mem_allocated_gpu0 = "{:.2f}".format(torch.cuda.max_memory_allocated(0)/1024**3) 
        pre_gen_max_mem_allocated_gpu1 = "{:.2f}".format(torch.cuda.max_memory_allocated(1)/1024**3)
        mem_stats_gpu0 = torch.cuda.memory_stats(0)
        mem_stats_gpu1 = torch.cuda.memory_stats(1)
        pre_gen_reserved_mem_gpu0 = "{:.2f}".format(mem_stats_gpu0["reserved_bytes.all.peak"]/1024**3)
        # pre_gen_active_mem_gpu0 = "{:.2f}".format(mem_stats_gpu0["active_bytes.all.peak"]/1024**3)
        pre_gen_reserved_mem_gpu1 = "{:.2f}".format(mem_stats_gpu1["reserved_bytes.all.peak"]/1024**3)
        prompt_for_chat.append({"role": "user", "content": prompt})    
        input_ids = tokenizer.apply_chat_template(prompt_for_chat, add_generation_prompt=True, return_tensors="pt").to(model.device)
        start_time = time.time() 
        if experiment_to_run == "sdpa" or experiment_to_run == "sdpa_torch.compile":
            with sdpa_kernel(SDPBackend.FLASH_ATTENTION):
                outputs = model.generate(input_ids, max_new_tokens=256, eos_token_id=terminators, do_sample=True, temperature=0.6, top_p=0.9,)
        else:
                outputs = model.generate(input_ids, max_new_tokens=256, eos_token_id=terminators, do_sample=True, temperature=0.6, top_p=0.9,)  
        # outputs = model.generate(input_ids, max_new_tokens=256, eos_token_id=terminators, do_sample=True, temperature=0.6, top_p=0.9,)
        end_time = time.time()
        latency = end_time - start_time
        response =  outputs[0][input_ids.shape[-1]:]
        llm_answer = tokenizer.decode(response, skip_special_tokens=True)
        prompt_for_chat.append({"role": "assistant", "content": llm_answer})
        post_gen_max_mem_allocated_gpu0 = "{:.2f}".format(torch.cuda.max_memory_allocated(0)/1024**3)
        post_gen_max_mem_allocated_gpu1 = "{:.2f}".format(torch.cuda.max_memory_allocated(1)/1024**3)
        mem_stats_gpu0 = torch.cuda.memory_stats()
        mem_stats_gpu1 = torch.cuda.memory_stats(1)
        post_gen_reserved_mem_gpu0 = "{:.2f}".format(mem_stats_gpu0["reserved_bytes.all.peak"]/1024**3)
        post_gen_reserved_mem_gpu1 = "{:.2f}".format(mem_stats_gpu1["reserved_bytes.all.peak"]/1024**3)
        # add the information to the dataframe using loc
        experiment_results_df.loc[len(experiment_results_df)] = ["default", prompt, input_ids.shape[-1], pre_gen_max_mem_allocated_gpu0, pre_gen_max_mem_allocated_gpu1, pre_gen_reserved_mem_gpu0, pre_gen_reserved_mem_gpu1, post_gen_max_mem_allocated_gpu0, post_gen_max_mem_allocated_gpu1, post_gen_reserved_mem_gpu0, post_gen_reserved_mem_gpu1, latency, llm_answer]

In [ ]:
experiment_results_df["llm_answer"]

In [ ]:
print(experiment_to_run)

In [ ]:
print(llm_answer)

In [ ]:
prompts_for_experiment = ["What is the firm period for Ognibene?"]
for prompt in prompts_for_experiment:
    pre_gen_max_mem_allocated_gpu0 = "{:.2f}".format(torch.cuda.max_memory_allocated(0)/1024**3) 
    pre_gen_max_mem_allocated_gpu1 = "{:.2f}".format(torch.cuda.max_memory_allocated(1)/1024**3)
    mem_stats_gpu0 = torch.cuda.memory_stats(0)
    mem_stats_gpu1 = torch.cuda.memory_stats(1)
    pre_gen_reserved_mem_gpu0 = "{:.2f}".format(mem_stats_gpu0["reserved_bytes.all.peak"]/1024**3)
    # pre_gen_active_mem_gpu0 = "{:.2f}".format(mem_stats_gpu0["active_bytes.all.peak"]/1024**3)
    pre_gen_reserved_mem_gpu1 = "{:.2f}".format(mem_stats_gpu1["reserved_bytes.all.peak"]/1024**3)
    prompt_for_chat.append({"role": "user", "content": prompt})    
    input_ids = tokenizer.apply_chat_template(prompt_for_chat, add_generation_prompt=True, return_tensors="pt").to(model.device)
    start_time = time.time()   
    outputs = model.generate(input_ids, max_new_tokens=256, eos_token_id=terminators, do_sample=True, temperature=0.6, top_p=0.9,)
    end_time = time.time()
    latency = end_time - start_time
    response =  outputs[0][input_ids.shape[-1]:]
    llm_answer = tokenizer.decode(response, skip_special_tokens=True)
    prompt_for_chat.append({"role": "assistant", "content": llm_answer})
    post_gen_max_mem_allocated_gpu0 = "{:.2f}".format(torch.cuda.max_memory_allocated(0)/1024**3)
    post_gen_max_mem_allocated_gpu1 = "{:.2f}".format(torch.cuda.max_memory_allocated(1)/1024**3)
    mem_stats_gpu0 = torch.cuda.memory_stats()
    mem_stats_gpu1 = torch.cuda.memory_stats(1)
    post_gen_reserved_mem_gpu0 = "{:.2f}".format(mem_stats_gpu0["reserved_bytes.all.peak"]/1024**3)
    post_gen_reserved_mem_gpu1 = "{:.2f}".format(mem_stats_gpu1["reserved_bytes.all.peak"]/1024**3)
    # add the information to the dataframe using loc
    experiment_results_df.loc[len(experiment_results_df)] = ["default", prompt, input_ids.shape[-1], pre_gen_max_mem_allocated_gpu0, pre_gen_max_mem_allocated_gpu1, pre_gen_reserved_mem_gpu0, pre_gen_reserved_mem_gpu1, post_gen_max_mem_allocated_gpu0, post_gen_max_mem_allocated_gpu1, post_gen_reserved_mem_gpu0, post_gen_reserved_mem_gpu1, latency, llm_answer]

    

In [ ]:
experiment_results_df

In [ ]:
# save the dataframe to a csv file
# experiment_results_df.to_csv("/home/vp899/projects/Long_Context_Chat/Long_Context_Chat/Output/experiment_results_expandable.csv", index=False)

In [ ]:
# print the main memory summary using torch.cuda.memory_summary()
print("Before delete")
print(torch.cuda.memory_summary())

# print maximum memory allocated and reserved memory for gpu0 and gpu1
print("Maximum memory allocated for GPU0: ", torch.cuda.max_memory_allocated(0)/1024**3)
print("Maximum memory allocated for GPU1: ", torch.cuda.max_memory_allocated(1)/1024**3)
mem_stats_gpu0 = torch.cuda.memory_stats(0)
mem_stats_gpu1 = torch.cuda.memory_stats(1)
print("Reserved memory for GPU0: ", mem_stats_gpu0["reserved_bytes.all.peak"]/1024**3)
print("Reserved memory for GPU1: ", mem_stats_gpu1["reserved_bytes.all.peak"]/1024**3)

In [ ]:
del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
print("Main memory stats after delete")
print(torch.cuda.memory_summary(1))
print("Maximum memory allocated for GPU0: ", torch.cuda.max_memory_allocated(0)/1024**3)
print("Maximum memory allocated for GPU1: ", torch.cuda.max_memory_allocated(1)/1024**3)
mem_stats_gpu0 = torch.cuda.memory_stats(0)
mem_stats_gpu1 = torch.cuda.memory_stats(1)
print("Reserved memory for GPU0: ", mem_stats_gpu0["reserved_bytes.all.peak"]/1024**3)
print("Reserved memory for GPU1: ", mem_stats_gpu1["reserved_bytes.all.peak"]/1024**3)
